## Ejemplo de webscraping del sitio inmuebles24.com

Este código realiza la extracción de la oferta y los precios para dos entidades: Ciudad de México y Yucatán. Se itera para poder obtener dos páginas por ciudad. La información recolectada aún deberá tratarse para poder obtener resultados y llegar a conclusiones.


In [70]:
#Librerías
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import re
import random
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [71]:
#Ciudades a buscar
ciudades=["ciudad-de-mexico", "yucatan"]

In [72]:
#Variables
url_basica="https://www.inmuebles24.com/inmuebles-en-venta-en-"
pagina="-pagina-"
url_final=".html"

In [73]:
# Lista de user agents
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
]

#Variables a extraer
data=[]

#Ciclo
for ciudad in ciudades:
    
    # Selección de user agent aleatorio
    user_agent = random.choice(user_agents)
    
    # Añadir user agent
    dcap = DesiredCapabilities.CHROME
    dcap["chrome.page.settings.userAgent"] = user_agent

    # Desactivar JavaScript
    options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.javascript": 2}
    options.add_experimental_option("prefs", prefs)

    # Se inicia el driver
    driver = webdriver.Chrome(service=Service(), desired_capabilities=dcap, options=options)

    for i in range(1,3):
        url=url_basica+ciudad+pagina+str(i)+url_final

        driver.get(url)
        time.sleep(5)
        soup=BeautifulSoup(driver.page_source,"html.parser")

        try:
            for div in soup.find_all('div', class_={'sc-1tt2vbg-3 eFJncP'}):
                oferta_element = div.find('h2', class_=re.compile('sc-i1odl-10'))
                precio_element = div.find('div', {'data-qa': 'POSTING_CARD_PRICE'})
                if oferta_element is not None and precio_element is not None:
                    data.append((oferta_element.text, precio_element.text, ciudad)) 
            print(f"Se escrapeó la página {i} de {ciudad}")
        except Exception as e:
            print(f"Ocurrió un error {url}: {e}")
        
        #Tiempo de espera para pasar a siguiente página
        time.sleep(random.uniform(5,10))

    # Cerremos el driver para pasar a la siguiente ciudad
    driver.quit()

Se escrapeó la página 1 de ciudad-de-mexico
Se escrapeó la página 2 de ciudad-de-mexico
Se escrapeó la página 1 de yucatan
Se escrapeó la página 2 de yucatan


In [74]:
#Se crea el dataframe
df=pd.DataFrame(data, columns=['oferta', 'precio', 'ciudad'])
df

,oferta,precio,ciudad
0,Departamento en Ampliación Nápoles con 2 Recám...,"MN 4,590,000",ciudad-de-mexico
1,Departamento en Venta Col. Portales Benito Jua...,"MN 3,192,094",ciudad-de-mexico
2,Ultimo Pent House con Amplio Roof Privado en C...,"MN 5,090,000",ciudad-de-mexico
3,Departamento en Granjas Modernas con 3 Recámar...,"MN 2,650,000",ciudad-de-mexico
4,Ventana Polanco Muy Bonito 3 Recamaras y Terraza,"MN 9,500,000",ciudad-de-mexico
5,Departamento Interior con Balcones y a Excelen...,"MN 2,020,000",ciudad-de-mexico
6,Casa en Venta en Bosque de Las Lomas,"MN 35,000,000",ciudad-de-mexico
7,"Casa en Venta en Itzapalapa, Cerca de Ermita","MN 3,800,000",ciudad-de-mexico
8,Departamento en Roma Sur,"MN 4,274,880",ciudad-de-mexico
9,Increíble Departamento en Venta Polanco Reside...,"MN 28,000,000",ciudad-de-mexico
